# Detección de Intrusiones y Análisis de Anomalías en Tráfico de Red mediante Técnicas Estadísticas

**Universidad de La Habana, MATCOM**  
**Curso:** Estadística 2025-2026  
**Proyecto Final:** Análisis Estadístico Aplicado a Seguridad Informática

---

## 1. Introducción al Proyecto

### 1.1 Contexto y Motivación

La seguridad informática es uno de los pilares fundamentales en la infraestructura tecnológica moderna. Los Sistemas de Detección de Intrusiones (IDS) tradicionales, basados en firmas conocidas, presentan limitaciones significativas frente a ataques emergentes o modificados (zero-day attacks). 

Este proyecto propone un enfoque complementario basado en **análisis estadístico del comportamiento del tráfico de red**, permitiendo identificar patrones anómalos sin depender exclusivamente de firmas previamente catalogadas. Este tipo de aproximación resulta especialmente relevante en entornos dinámicos donde los ataques evolucionan constantemente.

### 1.2 Dataset: NSL-KDD

#### 1.2.1 Contexto Histórico: La KDD Cup 1999

La **KDD Cup** (Knowledge Discovery and Data Mining) es una competición anual organizada por la ACM Special Interest Group on Knowledge Discovery and Data Mining (SIGKDD). En 1999, la competición se centró en el desafío de **detección de intrusiones en redes**, utilizando datos recopilados por el MIT Lincoln Laboratory en un entorno de red simulado.

El dataset original KDD Cup 1999 se convirtió en el benchmark más utilizado en investigación de ciberseguridad durante más de una década, siendo citado en miles de publicaciones académicas. Sin embargo, investigaciones posteriores identificaron **problemas metodológicos críticos**:

1. **Redundancia masiva**: ~78% de registros duplicados en entrenamiento, ~75% en prueba
2. **Sesgo de distribución**: Sobrerrepresentación artificial de ciertos ataques
3. **Facilidad artificial**: Modelos alcanzaban >99% de precisión por memorización de patrones redundantes

Estos problemas comprometían la validez de las conclusiones, llevando a resultados no generalizables a entornos reales.

---

#### 1.2.2 NSL-KDD: Versión Corregida y Validada

**Fuente:** [NSL-KDD en Kaggle](https://www.kaggle.com/datasets/hassan06/nslkdd) | [Paper original](http://www.unb.ca/cic/datasets/nsl.html) - Tavallaee et al. (2009)

El dataset **NSL-KDD** fue propuesto por investigadores del Canadian Institute for Cybersecurity (Universidad de New Brunswick) para resolver las limitaciones del KDD'99 original mediante:

✅ **Eliminación de registros duplicados** en train y test  
✅ **Balanceo mejorado** de distribuciones de ataque  
✅ **Niveles de dificultad** asignados a cada instancia  
✅ **Tamaño manejable** para pruebas reproducibles sin supercomputadoras

Pese a no ser perfecto, NSL-KDD sigue siendo el **estándar de facto en investigación académica** para benchmarking de algoritmos de detección de intrusiones, facilitando la comparabilidad entre estudios.

---

#### 1.2.3 Estructura del Dataset y Elección de Archivos

El dataset NSL-KDD en Kaggle contiene **cuatro archivos principales**:

| Archivo | Registros | Propósito |
|---------|-----------|-----------|
| **KDDTrain+.txt** | 125,973 | Entrenamiento completo |
| **KDDTrain+_20Percent.txt** | 25,192 | **Subconjunto estratificado (20%)** |
| **KDDTest+.txt** | 22,544 | **Evaluación final** |
| **KDDTest-21.txt** | 11,850 | Subset de test (21 categorías específicas) |

---

**🎯 Justificación de nuestra elección: `KDDTrain+_20Percent.txt` + `KDDTest+.txt`**

Utilizamos **KDDTrain+_20Percent** en lugar del dataset completo por las siguientes razones metodológicas:

1. **Representatividad garantizada**: El archivo 20% es una **muestra estratificada oficial** del conjunto completo, diseñada por los autores para mantener las mismas proporciones de clases y distribuciones estadísticas.

2. **Eficiencia computacional sin pérdida de validez**: Con 25,192 observaciones, el tamaño es suficiente para:
   - Cumplir supuestos de normalidad asintótica (n > 30 por grupo)
   - Detectar efectos pequeños con potencia estadística adecuada
   - Entrenar modelos complejos sin overfitting por exceso de datos redundantes

3. **Estándar académico**: Múltiples papers científicos utilizan este subset para comparabilidad de resultados (ver Dhanabal & Shantharajah, 2015; Revathi & Malathi, 2013).

4. **Tiempo de ejecución razonable**: Permite iteración rápida de análisis exploratorios y pruebas de hipótesis sin sacrificar rigor estadístico.

Utilizamos **KDDTest+** (no KDDTest-21) porque:
- Contiene la **distribución completa de 40 tipos de ataques**, incluyendo ataques no vistos en entrenamiento (realismo)
- KDDTest-21 es una versión restringida con solo 21 categorías específicas, útil para análisis focalizados pero menos representativa

---

#### 1.2.4 Características del Dataset

**Dimensiones:**
- **41 variables descriptoras del tráfico de red**
- **1 variable de etiquetado** (`attack_type`): indica el tipo de conexión observada
- **1 variable de metadatos** (`difficulty_level`): nivel de dificultad asignado por los autores
- **Conjunto de entrenamiento (20%):** 25,192 observaciones
- **Conjunto de prueba:** 22,544 observaciones

**Tipos de conexiones registradas (5 categorías principales):**

| Categoría | Descripción | Ejemplos de ataques específicos |
|-----------|-------------|--------------------------------|
| **Normal** | Tráfico de red legítimo | - |
| **DoS** | Denegación de servicio mediante sobrecarga | neptune, smurf, back, teardrop |
| **Probe** | Escaneo/reconocimiento de vulnerabilidades | portsweep, nmap, satan, ipsweep |
| **R2L** | Acceso no autorizado desde máquina remota | guess_passwd, ftp_write, warezmaster |
| **U2R** | Escalada de privilegios (user → root) | buffer_overflow, rootkit, loadmodule |

**Categorización de variables por naturaleza:**

- **Variables básicas (9):** Derivadas de cabeceras TCP/IP
  - *Ejemplos:* duration, protocol_type, service, flag, src_bytes, dst_bytes
  
- **Variables de contenido (13):** Información sobre el payload de la conexión
  - *Ejemplos:* num_failed_logins, root_shell, su_attempted, num_file_creations
  
- **Variables de tráfico temporal (9):** Estadísticas de ventanas de 2 segundos
  - *Ejemplos:* count, serror_rate, same_srv_rate, srv_count
  
- **Variables de tráfico basadas en host (10):** Estadísticas sobre las últimas 100 conexiones al mismo destino
  - *Ejemplos:* dst_host_count, dst_host_srv_count, dst_host_same_srv_rate

**Naturaleza de las variables:**
- **Continuas:** duration, src_bytes, dst_bytes (17 variables)
- **Discretas:** count, srv_count, num_failed_logins (21 variables)
- **Categóricas:** protocol_type (3 niveles), service (70 niveles), flag (11 niveles) (3 variables)
- **Binarias:** land, logged_in, root_shell, is_guest_login (17 variables)

---

#### 1.2.5 Limitaciones Conocidas (Transparencia Metodológica)

Pese a las mejoras sobre KDD'99, NSL-KDD presenta limitaciones que deben considerarse:

⚠️ **Desbalance de clases persistente**: DoS representa >50% del dataset  
⚠️ **Antigüedad de ataques**: Patrones de 1999, no reflejan amenazas modernas (ransomware, APTs)  
⚠️ **Entorno simulado**: Datos generados en laboratorio, no capturados de tráfico real  
⚠️ **Variables derivadas**: Algunas features asumen conocimiento previo del tipo de ataque

**Implicaciones para nuestro análisis:**
- Las conclusiones aplican a detección de **patrones generales** de comportamiento anómalo
- No pretendemos crear un IDS listo para producción, sino **demostrar la aplicabilidad de técnicas estadísticas** al problema
- Los resultados deben interpretarse en el contexto de un **ejercicio académico benchmark**, no como solución deployable

---

### 1.3 Carga e Inspección Inicial de Datos

En esta sección realizamos la carga de los archivos NSL-KDD, verificamos su integridad, creamos el mapeo de categorías de ataque y preparamos una muestra estratificada para visualizaciones complejas.

#### 1.3.1 Imports y Configuración del Entorno de Análisis

A continuación se importan las bibliotecas estándar de análisis de datos y se establece la configuración global de visualizaciones para mantener consistencia estética en todo el proyecto.

In [1]:
# Importación de librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import sklearn as sk
import scipy
import os
import statsmodels

# Configuración de visualizaciones
warnings.filterwarnings('ignore')
%matplotlib inline

# Estilo global para mantener consistencia en todas las visualizaciones
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 11
plt.rcParams['axes.titlesize'] = 13
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 10

# Paleta de colores consistente para categorías de ataque
# Se utilizará en todas las visualizaciones del proyecto
attack_colors = {
    'Normal': '#2ecc71',    # Verde - Tráfico legítimo
    'DoS': '#e74c3c',       # Rojo - Ataques de denegación de servicio
    'Probe': '#f39c12',     # Naranja - Ataques de reconocimiento
    'R2L': '#9b59b6',       # Morado - Acceso remoto no autorizado
    'U2R': '#34495e'        # Gris oscuro - Escalada de privilegios
}

print("✅ Librerías importadas y configuración de visualización establecida. \n Se utilizan bibliotecas estándar de análisis de datos y visualización ampliamente aceptadas en ciencia de datos, lo que garantiza reproducibilidad y compatibilidad.")

✅ Librerías importadas y configuración de visualización establecida. 
 Se utilizan bibliotecas estándar de análisis de datos y visualización ampliamente aceptadas en ciencia de datos, lo que garantiza reproducibilidad y compatibilidad.


#### 1.3.2 Carga de Archivos y Verificación de Estructura

In [2]:
# Definir los nombres de las columnas (43 columnas en total)
# Fuente: Documentación oficial NSL-KDD (Universidad de New Brunswick)
col_names = [
    # Variables básicas derivadas de cabeceras TCP/IP (9)
    "duration",           # Duración de la conexión en segundos
    "protocol_type",      # Protocolo: TCP, UDP, ICMP
    "service",            # Servicio de red destino (http, ftp, smtp, etc.)
    "flag",               # Estado de la conexión (SF, S0, REJ, etc.)
    "src_bytes",          # Bytes enviados desde origen a destino
    "dst_bytes",          # Bytes enviados desde destino a origen
    "land",               # 1 si origen y destino son iguales (ataque LAND)
    "wrong_fragment",     # Número de fragmentos incorrectos
    "urgent",             # Número de paquetes urgentes
    
    # Variables de contenido de la conexión (13)
    "hot",                # Número de indicadores "hot" (acceso a archivos críticos)
    "num_failed_logins",  # Número de intentos fallidos de login
    "logged_in",          # 1 si login exitoso
    "num_compromised",    # Número de condiciones "comprometidas"
    "root_shell",         # 1 si se obtuvo acceso root
    "su_attempted",       # 1 si comando "su root" fue intentado
    "num_root",           # Número de accesos root
    "num_file_creations", # Número de operaciones de creación de archivos
    "num_shells",         # Número de shells iniciados
    "num_access_files",   # Número de operaciones de acceso a archivos de control
    "num_outbound_cmds",  # Número de comandos FTP salientes (siempre 0 en dataset)
    "is_host_login",      # 1 si login es a host (no guest)
    "is_guest_login",     # 1 si login es de invitado
    
    # Variables de tráfico basadas en tiempo (ventana de 2 seg) (9)
    "count",              # Número de conexiones al mismo host en últimos 2 seg
    "srv_count",          # Número de conexiones al mismo servicio en últimos 2 seg
    "serror_rate",        # % de conexiones con errores SYN en count
    "srv_serror_rate",    # % de conexiones con errores SYN en srv_count
    "rerror_rate",        # % de conexiones con errores REJ en count
    "srv_rerror_rate",    # % de conexiones con errores REJ en srv_count
    "same_srv_rate",      # % de conexiones al mismo servicio en count
    "diff_srv_rate",      # % de conexiones a diferentes servicios en count
    "srv_diff_host_rate", # % de conexiones a diferentes hosts en srv_count
    
    # Variables de tráfico basadas en host (últimas 100 conexiones) (10)
    "dst_host_count",              # Conexiones al mismo host destino
    "dst_host_srv_count",          # Conexiones al mismo servicio en host destino
    "dst_host_same_srv_rate",      # % conexiones al mismo servicio
    "dst_host_diff_srv_rate",      # % conexiones a diferentes servicios
    "dst_host_same_src_port_rate", # % conexiones desde mismo puerto origen
    "dst_host_srv_diff_host_rate", # % conexiones desde diferentes hosts
    "dst_host_serror_rate",        # % errores SYN en dst_host_count
    "dst_host_srv_serror_rate",    # % errores SYN en dst_host_srv_count
    "dst_host_rerror_rate",        # % errores REJ en dst_host_count
    "dst_host_srv_rerror_rate",    # % errores REJ en dst_host_srv_count
    
    # Variables objetivo y metadatos (2)
    "attack_type",        # Etiqueta: tipo específico de ataque o "normal"
    "difficulty_level"    # Nivel de dificultad (0-21) asignado por los autores
]

# Cargar los datasets
# Justificación de archivos seleccionados (ver sección 1.2.3):
# - KDDTrain+_20Percent: Muestra estratificada oficial (25,192 obs)
# - KDDTest+: Conjunto completo de evaluación (22,544 obs)

print("📂 Cargando archivos NSL-KDD...")
train_df = pd.read_csv('Data/KDDTrain+_20Percent.txt', 
                       names=col_names, 
                       header=None)

test_df = pd.read_csv('Data/KDDTest+.txt', 
                      names=col_names, 
                      header=None)

# Crear variable binaria para análisis de dos grupos (Normal vs. Ataque)
# Nota: Su uso específico se determinará tras el análisis exploratorio
train_df['is_attack'] = (train_df['attack_type'] != 'normal').astype(int)
test_df['is_attack'] = (test_df['attack_type'] != 'normal').astype(int)

# Mostrar información básica
print(f"\n✅ Datasets cargados exitosamente:")
print(f"   📊 Entrenamiento: {train_df.shape[0]:,} observaciones × {train_df.shape[1]} variables")
print(f"   📊 Prueba:        {test_df.shape[0]:,} observaciones × {test_df.shape[1]} variables")
print(f"\n📌 Distribución binaria en ENTRENAMIENTO:")
print(f"   Normal:  {(train_df['is_attack'] == 0).sum():,} ({(train_df['is_attack'] == 0).sum() / len(train_df) * 100:.2f}%)")
print(f"   Ataques: {(train_df['is_attack'] == 1).sum():,} ({(train_df['is_attack'] == 1).sum() / len(train_df) * 100:.2f}%)")

📂 Cargando archivos NSL-KDD...

✅ Datasets cargados exitosamente:
   📊 Entrenamiento: 25,192 observaciones × 44 variables
   📊 Prueba:        22,544 observaciones × 44 variables

📌 Distribución binaria en ENTRENAMIENTO:
   Normal:  13,449 (53.39%)
   Ataques: 11,743 (46.61%)


#### 1.3.3 Mapeo de Categorías de Ataque

El dataset NSL-KDD contiene 40 tipos de ataques específicos que se agrupan en 4 categorías principales más la clase normal. A continuación se realiza el mapeo oficial según la documentación del Canadian Institute for Cybersecurity:
- **Normal:** Tráfico de red legítimo
- **DoS (Denial of Service):** Ataques que buscan denegar el servicio mediante sobrecarga de recursos
- **Probe (Probing/Scanning):** Ataques de reconocimiento que escanean la red en busca de vulnerabilidades
- **R2L (Remote to Local):** Intentos de acceso no autorizado desde una máquina remota
- **U2R (User to Root):** Intentos de escalada de privilegios de usuario normal a superusuario

Este agrupamiento reduce la complejidad del problema y permite realizar inferencias estadísticas más robustas sobre patrones generales de ataque.

In [3]:
# Diccionario oficial de mapeo de ataques específicos a categorías generales
# Fuente: Documentación oficial NSL-KDD (Canadian Institute for Cybersecurity)
attack_category_mapping = {
    # Tráfico Normal
    'normal': 'Normal',
    
    # DoS (Denial of Service) - Ataques de denegación de servicio
    'back': 'DoS', 'land': 'DoS', 'neptune': 'DoS', 'pod': 'DoS',
    'smurf': 'DoS', 'teardrop': 'DoS', 'mailbomb': 'DoS', 'apache2': 'DoS',
    'processtable': 'DoS', 'udpstorm': 'DoS',
    
    # Probe (Probing/Scanning) - Ataques de reconocimiento
    'ipsweep': 'Probe', 'nmap': 'Probe', 'portsweep': 'Probe',
    'satan': 'Probe', 'mscan': 'Probe', 'saint': 'Probe',
    
    # R2L (Remote to Local) - Acceso no autorizado desde máquina remota
    'ftp_write': 'R2L', 'guess_passwd': 'R2L', 'imap': 'R2L',
    'multihop': 'R2L', 'phf': 'R2L', 'spy': 'R2L',
    'warezclient': 'R2L', 'warezmaster': 'R2L', 'sendmail': 'R2L',
    'named': 'R2L', 'snmpgetattack': 'R2L', 'snmpguess': 'R2L',
    'xlock': 'R2L', 'xsnoop': 'R2L', 'worm': 'R2L', 'xterm': 'R2L', 
    
    # U2R (User to Root) - Escalada de privilegios
    'buffer_overflow': 'U2R', 'loadmodule': 'U2R', 'perl': 'U2R',
    'rootkit': 'U2R', 'httptunnel': 'U2R', 'ps': 'U2R', 'sqlattack': 'U2R'
}

# Aplicar mapeo a ambos datasets
train_df['attack_category'] = train_df['attack_type'].map(attack_category_mapping)
test_df['attack_category'] = test_df['attack_type'].map(attack_category_mapping)

# Verificar que no hay valores sin mapear
print("🔍 Verificación de mapeo de categorías:")
unmapped_train = train_df['attack_category'].isna().sum()
unmapped_test = test_df['attack_category'].isna().sum()
print(f"   Valores sin mapear en train: {unmapped_train}")
print(f"   Valores sin mapear en test: {unmapped_test}")

if unmapped_test > 0:
    print(f"\n⚠️ ADVERTENCIA: Hay {unmapped_test} ataques en test sin categoría asignada.")

# Crear orden categórico para visualizaciones consistentes
category_order = ['Normal', 'DoS', 'Probe', 'R2L', 'U2R']
train_df['attack_category'] = pd.Categorical(
    train_df['attack_category'], 
    categories=category_order, 
    ordered=True
)
test_df['attack_category'] = pd.Categorical(
    test_df['attack_category'], 
    categories=category_order, 
    ordered=True
)

# Mostrar distribución de categorías
print("\n📊 Distribución de categorías en ENTRENAMIENTO:")
category_dist_train = train_df['attack_category'].value_counts()
category_pct_train = train_df['attack_category'].value_counts(normalize=True) * 100
category_summary_train = pd.DataFrame({
    'Frecuencia': category_dist_train,
    'Porcentaje': category_pct_train.round(2)
})
print(category_summary_train)

print("\n📊 Distribución de categorías en PRUEBA:")
category_dist_test = test_df[test_df['attack_category'].notna()]['attack_category'].value_counts()
category_pct_test = test_df[test_df['attack_category'].notna()]['attack_category'].value_counts(normalize=True) * 100
category_summary_test = pd.DataFrame({
    'Frecuencia': category_dist_test,
    'Porcentaje': category_pct_test.round(2)
})
print(category_summary_test)

print("\n✅ Mapeo de categorías completado exitosamente")

🔍 Verificación de mapeo de categorías:
   Valores sin mapear en train: 0
   Valores sin mapear en test: 0

📊 Distribución de categorías en ENTRENAMIENTO:
                 Frecuencia  Porcentaje
attack_category                        
Normal                13449       53.39
DoS                    9234       36.65
Probe                  2289        9.09
R2L                     209        0.83
U2R                      11        0.04

📊 Distribución de categorías en PRUEBA:
                 Frecuencia  Porcentaje
attack_category                        
Normal                 9711       43.08
DoS                    7458       33.08
R2L                    2767       12.27
Probe                  2421       10.74
U2R                     187        0.83

✅ Mapeo de categorías completado exitosamente


#### 1.3.4 Preparación de Muestra Estratificada para Visualizaciones

Para optimizar el rendimiento de visualizaciones complejas (como scatterplot matrices y pairplots), crearemos una muestra estratificada de 5,000 observaciones que mantenga las proporciones originales de cada categoría de ataque. 

**Nota importante:** Esta muestra se utilizará **exclusivamente para visualizaciones**. Todos los análisis estadísticos (correlaciones, pruebas de hipótesis, modelos) se realizarán sobre el dataset completo.

In [4]:
# Configuración del muestreo estratificado
sample_size = 5000
random_state = 42  # Para reproducibilidad

# Crear muestra estratificada manteniendo proporciones de cada categoría
train_sample = train_df.groupby('attack_category', group_keys=False).apply(
    lambda x: x.sample(
        n=min(len(x), int(sample_size * len(x) / len(train_df))),
        random_state=random_state
    )
).reset_index(drop=True)

print(f"📊 Muestra estratificada creada: {len(train_sample):,} observaciones")
print(f"\n✅ Verificación de estratificación:")

# Comparar proporciones originales vs. muestra
comparison = pd.DataFrame({
    'Original (%)': train_df['attack_category'].value_counts(normalize=True).sort_index() * 100,
    'Muestra (%)': train_sample['attack_category'].value_counts(normalize=True).sort_index() * 100
})
comparison['Diferencia (pp)'] = (comparison['Muestra (%)'] - comparison['Original (%)']).abs()
print(comparison.round(2))

print("\n💡 Esta muestra se usará únicamente para visualizaciones pesadas.")
print("   Los análisis estadísticos utilizarán el dataset completo.")

📊 Muestra estratificada creada: 4,998 observaciones

✅ Verificación de estratificación:
                 Original (%)  Muestra (%)  Diferencia (pp)
attack_category                                            
Normal                  53.39        53.40             0.02
DoS                     36.65        36.65             0.00
Probe                    9.09         9.08             0.00
R2L                      0.83         0.82             0.01
U2R                      0.04         0.04             0.00

💡 Esta muestra se usará únicamente para visualizaciones pesadas.
   Los análisis estadísticos utilizarán el dataset completo.
